# Materials Calculator: Version 2

## Instructions:

1. Go to the "Items tab" of your Excel Sheet working copy.
2. Save as > File Format > CSV
3. Drag and drop the file into the menu on the left + right click it to "Copy Path"
4. Run the notebook and insert the pathname when prompted
5. Insert the item quantities, one by one, as prompted
6. Tadaaaa!


Nas path - /Users/anastasiakuzmich/Desktop/YAKU_Season 1_Cost Sheet_Nas_Reformat.csv

In [25]:
# nylon_list = ["Atomic Blue Waterproof nylon", "Ripstop Nylon", "Red Waterproof Nylon"]

# costs = costs[costs["Cost Name"].isin(nylon_list)]
# costs



,I-Code,Item Name,C-Code,Cost Name,Dye_Type,Description,Yield,UOM,Supplier,Cost Per,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
231,YS1_GIL_S_RED,S-Gilet (Red),M0022,Red Waterproof Nylon,NaN,NaN,3.0,m,TBC,£-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
### CLEANING

costs = costs.dropna(how='all', axis=1)
costs = costs.dropna(how='all', axis=0)

# hide_toggle()

In [27]:
## Drop unnamed columns

to_drop = costs.columns[costs.columns.str.startswith('Unnamed')]
costs.drop(to_drop, axis=1, inplace=True)

In [28]:
costs["Description"] = costs["Description"].fillna("")
costs['Item Name'] = costs['Item Name'].str.strip()
costs

,I-Code,Item Name,C-Code,Cost Name,Yield,UOM,Supplier,Cost Per,Unit Cost,Cost Label,Cost Sublabel,Season
231,YS1_GIL_S_RED,S-Gilet (Red),M0022,Red Waterproof Nylon,3.0,m,TBC,£-,£-,Materials,Fabric,YS1


In [29]:
for column in costs.columns:
    costs[column] = costs[column].fillna("")

In [30]:
### SEASON FILTER
    
season_filter = input("Would you like to filter by season? (y/n)")

if season_filter == "n":
    pass
elif season_filter == "y":
    season_selector = input("Enter M for mainline, R for resort or B for both:")
    if season_selector.lower() == "m":
        costs = costs[costs["Season"] == "YS1"]
        costs = pd.DataFrame(costs)
    elif season_selector.lower() == "r":
        costs = costs[costs["Season"] == "YR1"]
        costs = pd.DataFrame(costs)
    else:
        pass

Would you like to filter by season? (y/n)n


In [31]:
### CLEANING

new_costs = []

for cost_name, item_name, description, uom, dye_colour in zip(costs['Cost Name'], costs['Item Name'], costs["Description"], costs["UOM"], costs["Dye_Type"]):

    Renaming dyes for clarity on colour
    if cost_name == "Dye":
        new_costs.append(str(cost_name + " - " + dye_colour))
        
    # Replacing question marks with Item Names for Clarity
    elif "?" in cost_name:
        replacement = "for" + str(item_name)
        new_costs.append(cost_name.replace('?"', replacement))
        
    # Adding the material kind to YKK zips
    elif "YKK" in cost_name:
        new_costs.append(str(cost_name + " (" + description + ")"))
    
    # Adding the size to the Sublimation Dyes
    elif cost_name == "Sublimation Dye":
        new_costs.append(str(cost_name + " (" + uom + ")"))
        
    elif cost_name == "Sublimation Print":
        new_costs.append(str(cost_name + " - " + dye_colour))
    
    else:
        new_costs.append(cost_name)
        
for cost_name, item_name, uom in zip(costs['Cost Name'], costs['Item Name'], costs["UOM"]):
    
    # Renaming dyes for clarity on colour
    if cost_name == "Dye":
        new_costs.append(str(cost_name + " - " + dye_colour))
        
    # Replacing question marks with Item Names for Clarity
    elif "?" in cost_name:
        replacement = "for" + str(item_name)
        new_costs.append(cost_name.replace('?"', replacement))
        
#     # Adding the material kind to YKK zips
#     elif "YKK" in cost_name:
#         new_costs.append(str(cost_name + " (" + description + ")"))
    
    # Adding the size to the Sublimation Dyes
    elif cost_name == "Sublimation Dye":
        new_costs.append(str(cost_name + " (" + uom + ")"))
        
    elif cost_name == "Sublimation Print":
        new_costs.append(str(cost_name + " - " + dye_colour))
    
    else:
        new_costs.append(cost_name)

costs["costs"] = new_costs

In [32]:
new_UOM = []

for uom in costs["UOM"]:
    if "A" in uom:
        new_UOM.append("pcs")
    else:
        new_UOM.append(uom)

costs["UOM"] = new_UOM

material_costs = costs[costs["Cost Label"] == "Materials"]

material_costs = material_costs.drop(["C-Code", "Cost Name", "I-Code", "Unit Cost", "Cost Label", "Cost Sublabel"], axis=1)
# material_costs = material_costs.drop(["C-Code", "Cost Name", "I-Code", "Unit Cost", "Cost Label", "Cost Sublabel", "Description"], axis=1)

material_costs

,Item Name,Yield,UOM,Supplier,Cost Per,Season,costs
231,S-Gilet (Red),3.0,m,TBC,£-,YS1,Red Waterproof Nylon


In [33]:
cost_per = []

for cost in material_costs["Cost Per"]:
    if "-" in cost:
        cost_per.append("0")
    elif "£" in cost:
        cost_per.append(cost.replace("£", ""))
    else:
        cost_per.append(cost)

cost_per = [float(i) for i in cost_per]
material_costs["Cost Per"] = cost_per

del new_costs, new_UOM, cost_per

material_costs = material_costs.rename(columns={"Yield": "Units", "Cost Per": "Cost Per Unit", "costs" : "Material"})

costs = pd.DataFrame(material_costs)

costs

,Item Name,Units,UOM,Supplier,Cost Per Unit,Season,Material
231,S-Gilet (Red),3.0,m,TBC,0.0,YS1,Red Waterproof Nylon


In [34]:
### USER INPUT - ORDER QUANTITY

item_names = {}

for item_name in costs["Item Name"].unique():
    item_names[item_name] = 0
    
for key, value in item_names.items():
    item_names[key] = int(input("How many " + key + "?"))

How many S-Gilet (Red)?7


In [35]:
##### CALCULATIONS

costs['Quantity'] = costs['Item Name'].map(item_names)

material_costs = costs.copy()

material_costs["Total Units"] = material_costs["Units"] * material_costs["Quantity"]

material_costs["Total Cost"] = material_costs["Total Units"] * material_costs["Cost Per Unit"]

material_costs.head()

,Item Name,Units,UOM,Supplier,Cost Per Unit,Season,Material,Quantity,Total Units,Total Cost
231,S-Gilet (Red),3.0,m,TBC,0.0,YS1,Red Waterproof Nylon,7,21.0,0.0


In [36]:
del material_costs["Item Name"]

In [37]:
### OUTPUT

print("Your Quantity Request:")

print("\n")

for key, value in item_names.items():
    print(str(value) + " " + str(key) + "s")
    
print("\n")

print("This order will cost you", round(material_costs["Total Cost"].sum(),2), "in total")
print("\n")
print("Your price breakdown is:")

Your Quantity Request:


7 S-Gilet (Red)s


This order will cost you 0.0 in total


Your price breakdown is:


In [38]:


# mat_and_uom = {}

# for material, uom in zip(material_costs['Material'], material_costs['UOM']):
#     if material not in mat_and_uom.keys():
#         mat_and_uom[material] = uom
        
# mat_and_sup = {}

# for material, sup in zip(material_costs['Material'], material_costs['Supplier']):
#     if material not in mat_and_sup.keys():
#         mat_and_sup[material] = sup

# output_costs['Units of Measure'] = output_costs['Material'].map(mat_and_uom)
# output_costs['Supplier'] = output_costs['Material'].map(mat_and_sup)

# output_costs = output_costs[["Material", "Total Units", "Units of Measure", "Total Cost", "Supplier"]].sort_values(by="Supplier")

output_costs

,Material,Total Units,Total Cost
0,Red Waterproof Nylon,21.0,0.0
